In [1]:
print "Monitoring URL: http://hadoop-lhr4-rm1.prod.booking.com:8088/proxy/" + spark.sparkContext.applicationId + '/stages'

Monitoring URL: http://hadoop-lhr4-rm1.prod.booking.com:8088/proxy/application_1512643462013_260449/stages


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import *

/opt/blue-python/2.7/lib/python2.7/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [10]:
train = pd.read_csv('train.csv')

In [11]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [13]:
train = train.drop('PassengerId',axis=1)

In [14]:
train = train.drop('Cabin',axis=1)

In [19]:
train = train.drop('Ticket',axis=1)

In [20]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,29.699118,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,29.699118,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,29.699118,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,29.699118,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,29.699118,0,0,8.0500,S


In [17]:
train['Age']=train['Age'].mean()

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [21]:
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [23]:
train = pd.concat([train,sex,embark],axis=1)

In [24]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,male,Q,S
0,0,3,"Braund, Mr. Owen Harris",male,29.699118,1,0,7.2500,S,1,0,1
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,29.699118,1,0,71.2833,C,0,0,0
2,1,3,"Heikkinen, Miss. Laina",female,29.699118,0,0,7.9250,S,0,0,1
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,29.699118,1,0,53.1000,S,0,0,1
4,0,3,"Allen, Mr. William Henry",male,29.699118,0,0,8.0500,S,1,0,1


In [26]:
train.drop('Name',axis=1,inplace=True)

In [27]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,male,Q,S
0,0,3,male,29.699118,1,0,7.2500,S,1,0,1
1,1,1,female,29.699118,1,0,71.2833,C,0,0,0
2,1,3,female,29.699118,0,0,7.9250,S,0,0,1
3,1,1,female,29.699118,1,0,53.1000,S,0,0,1
4,0,3,male,29.699118,0,0,8.0500,S,1,0,1


In [28]:
train.drop('Sex',axis=1,inplace=True)

In [29]:
train.drop('Embarked',axis=1,inplace=True)

In [30]:
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,0,3,29.699118,1,0,7.2500,1,0,1
1,1,1,29.699118,1,0,71.2833,0,0,0
2,1,3,29.699118,0,0,7.9250,0,0,1
3,1,1,29.699118,1,0,53.1000,0,0,1
4,0,3,29.699118,0,0,8.0500,1,0,1


In [32]:
type(train.corr())

pandas.core.frame.DataFrame

In [34]:
train.corr().round(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
Survived,1.00,-0.34,0.0,-0.04,0.08,0.26,-0.54,0.00,-0.16
Pclass,-0.34,1.00,-0.0,0.08,0.02,-0.55,0.13,0.22,0.08
Age,0.00,-0.00,1.0,0.00,0.00,0.00,0.00,0.00,-0.00
SibSp,-0.04,0.08,0.0,1.00,0.41,0.16,-0.11,-0.03,0.07
Parch,0.08,0.02,0.0,0.41,1.00,0.22,-0.25,-0.08,0.06
Fare,0.26,-0.55,0.0,0.16,0.22,1.00,-0.18,-0.12,-0.17
male,-0.54,0.13,0.0,-0.11,-0.25,-0.18,1.00,-0.07,0.13
Q,0.00,0.22,0.0,-0.03,-0.08,-0.12,-0.07,1.00,-0.50
S,-0.16,0.08,-0.0,0.07,0.06,-0.17,0.13,-0.50,1.00


In [36]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
logmodel = LogisticRegression()

In [39]:
%pwd

u'/sparkshell_nfs/public/yyoung/Training/Titanic'